In [ ]:
!pip install resampy

In [ ]:

# import matplotlib.pyplot as plt
# import tensorflow as tf
# from matplotlib.pyplot import specgram
# import keras
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Embedding
# from keras.layers import LSTM
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical
# from keras.layers import Input, Flatten, Dropout, Activation
# from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
# from keras.models import Model
# from keras.callbacks import ModelCheckpoint
# from sklearn.metrics import confusion_matrix
# from tqdm import tqdm
import os
import pandas as pd
# import librosa
# import glob
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification

import librosa
import librosa.display
import numpy as np

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
raw_data_path = '/content/drive/MyDrive/CSCI-534/RawData/'

In [ ]:
from keras.models import model_from_json
json_file = open('/content/drive/MyDrive/CSCI-534/Emotion_Voice_Detection_Model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/CSCI-534/Emotion_Voice_Detection_Model.h5")

In [ ]:
sample_video_file_path = "/content/drive/MyDrive/CSCI-534/RawData/03-02-01-01-01-01-01.wav"
# data, sampling_rate = librosa.load(sample_video_file_path)

In [ ]:
def load_audio(file_path, target_sr=16000):
    # Load the file with librosa, which automatically resamples to the target_sr
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    return audio

In [ ]:
import tensorflow_hub as hub
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)
waveform = load_audio(sample_video_file_path)
scores, embeddings, spectrogram = yamnet_model(waveform)
features = np.mean(embeddings, axis=0)
df = pd.DataFrame(columns=['feature'])
df.loc[0] = [features]

In [ ]:
df3 = pd.DataFrame(df['feature'].values.tolist())

In [ ]:
type(df3)
# X = np.array(df3)
X = np.expand_dims(np.array(df3), axis=2)

In [ ]:
preds = loaded_model.predict(X, verbose=1)

1/1 [==============================] - 1s 814ms/step


In [ ]:
preds

array([[1.2783718e-04, 4.7711807e-04, 7.5004027e-05, 4.5732140e-05,
        1.3415363e-04, 7.6861060e-03, 2.7105597e-01, 4.5390546e-02,
        8.3671436e-02, 5.9133613e-01]], dtype=float32)

In [ ]:
id2label = ['female_angry', 'female_calm', 'female_fearful', 'female_happy',
       'female_sad', 'male_angry', 'male_calm', 'male_fearful',
       'male_happy', 'male_sad']